In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from icecream import ic
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from xgboost import XGBClassifier

import random
import numpy as np
import torch

from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, confusion_matrix
import numpy as np
import time

from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import optuna

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## General AMP

### Data

In [7]:
adam_df = pd.read_csv("../data/naturalAMPs_APD2024a-ADAM.csv")
# uniprot_df = pd.read_csv("../data/uniprotkb_length_10_TO_80_NOT_antimicro_2025_04_14.fasta.csv")
# uniprot_df1 = pd.read_csv("../data/uniprotkb_length_10_TO_80_NOT_antimicro_2025_04_14.fasta1.csv")
# uniprot_df = pd.concat([uniprot_df, uniprot_df1], ignore_index=True)

#### raw data gen

In [19]:
import torch
import pandas as pd
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Load only positive (AMP) sequences
adam_df = pd.read_csv("../data/naturalAMPs_APD2024a-ADAM.csv")

unique_letters = set(''.join(adam_df["Sequence"]))
print(unique_letters)
print(len(unique_letters))
print(f"Number of sequences after filtering: {len(adam_df)}")
adam_df = adam_df.drop_duplicates(subset='Sequence')
tb_df = pd.read_csv('../data/all_seq702.csv')
adam_df = adam_df[~adam_df['Sequence'].isin(tb_df['Sequences'])]

generation_seqs = adam_df["Sequence"].reset_index(drop=True)

import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

def one_hot_torch(seq: str, dtype=torch.float32):
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    seq_bytes = torch.ByteTensor(list(bytes(seq, "utf-8")))
    aa_bytes = torch.ByteTensor(list(bytes(amino_acids, "utf-8")))
    arr = torch.zeros(len(amino_acids), len(seq_bytes), dtype=dtype)
    for i, aa in enumerate(aa_bytes):
        arr[i, seq_bytes == aa] = 1
    return arr

class GenerativeSequenceDataset(Dataset):
    def __init__(self, sequences, one_hot_dtype=torch.float32):
        self.sequences = sequences
        self.one_hot_dtype = one_hot_dtype

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences.iloc[idx]
        input_seq = seq[:-1]  # all residues except the last
        target_seq = seq[1:]  # all residues except the first
        length = len(input_seq.replace("X", ""))  # unpadded length
        input_one_hot = one_hot_torch(input_seq, dtype=self.one_hot_dtype)
        target_one_hot = one_hot_torch(target_seq, dtype=self.one_hot_dtype)
        # target_indices = torch.tensor(["ACDEFGHIKLMNPQRSTVWY".index(res) for res in target_seq], dtype=torch.long)
        return input_one_hot, target_one_hot, length

def generative_collate_and_pack(batch):
    sequences, targets, lengths = zip(*batch)

    lengths = torch.tensor(lengths)
    sorted_indices = torch.argsort(lengths, descending=True)
    sequences = [sequences[i] for i in sorted_indices]
    targets = [targets[i] for i in sorted_indices]
    lengths = lengths[sorted_indices]

    sequences = [seq.T for seq in sequences]  # transpose to [seq_len, features]
    targets = [tgt.T for tgt in targets]      # transpose targets as well

    padded_seqs = pad_sequence(sequences, batch_first=False)
    padded_targets = pad_sequence(targets, batch_first=False)

    packed_input = pack_padded_sequence(padded_seqs, lengths.cpu(), batch_first=False)
    packed_target = pack_padded_sequence(padded_targets, lengths.cpu(), batch_first=False)

    return packed_input, packed_target, lengths


# Train/val/test split
train_seqs, test_seqs = train_test_split(generation_seqs, test_size=0.3, random_state=42)
val_seqs, test_seqs = train_test_split(test_seqs, test_size=0.5, random_state=42)

train_dataset = GenerativeSequenceDataset(train_seqs)
val_dataset = GenerativeSequenceDataset(val_seqs)
test_dataset = GenerativeSequenceDataset(test_seqs)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=generative_collate_and_pack)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=generative_collate_and_pack)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=generative_collate_and_pack)

# Dataset sizes
dataset_sizes = {
    "Train": len(train_dataset),
    "Validation": len(val_dataset),
    "Test": len(test_dataset)
}
print("Dataset sizes:", dataset_sizes)

for x, y, l in train_loader:
    print("Input shape:", x.data.shape)  # [L, B, 20]
    print("Target shape:", y.data.shape)  # [L, B, 20]
    print("Lengths:", y.batch_sizes)  # Lengths of sequences in the batch
    break

{'W', 'L', 'Q', 'F', 'K', 'C', 'H', 'T', 'E', 'P', 'R', 'V', 'D', 'G', 'N', 'S', 'Y', 'I', 'M', 'A'}
20
Number of sequences after filtering: 3306
Dataset sizes: {'Train': 2276, 'Validation': 488, 'Test': 488}
Input shape: torch.Size([2495, 20])
Target shape: torch.Size([2495, 20])
Lengths: tensor([64, 64, 64, 64, 64, 64, 63, 63, 63, 63, 63, 63, 61, 61, 60, 58, 55, 54,
        52, 52, 49, 48, 48, 47, 46, 45, 43, 41, 37, 35, 33, 29, 27, 25, 25, 23,
        23, 23, 22, 21, 21, 20, 18, 17, 16, 14, 14, 12, 10, 10,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  8,  8,  7,  7,  6,  6,  6,  6,  6,  6,  6,  6,  5,  5,  5,  5,
         5,  5,  5,  5,  5,  5,  5,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  2,  2,  2,  2,  2,  2,  2,  2])


#### sliding window data gen

In [20]:
# # Re-import required libraries after environment reset
# import torch
# import pandas as pd
# from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split

# # Load only positive (AMP) sequences
# adam_df = pd.read_csv("../data/naturalAMPs_APD2024a-ADAM.csv")

# # Clean non-standard amino acids
# unique_letters = set(''.join(adam_df["Sequence"]))
# amino_acids = "ACDEFGHIKLMNPQRSTVWY"
# # non_standard_amino_acids = set(unique_letters) - set(amino_acids)
# # adam_df = adam_df[~adam_df["Sequence"].str.contains('|'.join(non_standard_amino_acids))]

# # Apply sliding window to generate fragments
# def generate_fragments(sequences, window_size=15, stride=5):
#     fragments = []
#     for seq in sequences:
#         for start in range(0, len(seq) - window_size + 1, stride):
#             fragment = seq[start:start + window_size]
#             fragments.append(fragment)
#     return fragments

# generation_fragments = generate_fragments(adam_df["Sequence"].tolist())

# # Define one-hot encoding function
# def one_hot_torch(seq: str, dtype=torch.float32):
#     amino_acids = "ACDEFGHIKLMNPQRSTVWY"
#     seq_bytes = torch.ByteTensor(list(bytes(seq, "utf-8")))
#     aa_bytes = torch.ByteTensor(list(bytes(amino_acids, "utf-8")))
#     arr = torch.zeros(len(amino_acids), len(seq_bytes), dtype=dtype)
#     for i, aa in enumerate(aa_bytes):
#         arr[i, seq_bytes == aa] = 1
#     return arr

# # Dataset using one-hot encoding for generative modeling
# class AMPGenerationOneHotDataset(Dataset):
#     def __init__(self, sequences):
#         self.sequences = sequences

#     def __len__(self):
#         return len(self.sequences)

#     def __getitem__(self, idx):
#         seq = self.sequences[idx]
#         input_seq = seq[:-1]  # all residues except the last
#         target_seq = seq[1:]  # all residues except the first
#         input_one_hot = one_hot_torch(input_seq)  # shape: [20, seq_len - 1]
#         target_one_hot = one_hot_torch(target_seq)  # shape: [20, seq_len - 1]
#         length = input_one_hot.shape[1]
#         return input_one_hot, target_one_hot, length

# # Collate function for packing one-hot sequences
# def collate_and_pack_for_generation(batch):
#     sequences, targets, lengths = zip(*batch)
#     lengths = torch.tensor(lengths)

#     # Sort by length (required for packing)
#     sorted_indices = torch.argsort(lengths, descending=True)
#     sequences = [sequences[i] for i in sorted_indices]
#     targets = [targets[i] for i in sorted_indices]
#     lengths = lengths[sorted_indices]

#     # Transpose each to [L, 20] and pad
#     sequences = [seq.T for seq in sequences]  # from [20, L] to [L, 20]
#     targets = [tgt.T for tgt in targets]      # from [20, L] to [L, 20]

#     padded_seqs = pad_sequence(sequences, batch_first=False)  # [L, B, 20]
#     padded_targets = pad_sequence(targets, batch_first=False)  # [L, B, 20]

#     packed_input = pack_padded_sequence(padded_seqs, lengths.cpu(), batch_first=False)
#     packed_target = pack_padded_sequence(padded_targets, lengths.cpu(), batch_first=False)

#     return packed_input, packed_target, lengths

# # Train/val/test split
# train_seqs, test_seqs = train_test_split(generation_fragments, test_size=0.3, random_state=42)
# val_seqs, test_seqs = train_test_split(test_seqs, test_size=0.5, random_state=42)

# train_dataset = AMPGenerationOneHotDataset(train_seqs)
# val_dataset = AMPGenerationOneHotDataset(val_seqs)
# test_dataset = AMPGenerationOneHotDataset(test_seqs)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_and_pack_for_generation)
# val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=collate_and_pack_for_generation)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_and_pack_for_generation)

# # Dataset sizes for verification
# dataset_sizes = {
#     "Train": len(train_dataset),
#     "Validation": len(val_dataset),
#     "Test": len(test_dataset)
# }
# print("Dataset sizes:", dataset_sizes)

# for x, y, l in train_loader:
#     print("Input shape:", x.data.shape)  # [L, B, 20]
#     print("Target shape:", y.data.shape)  # [L, B, 20]
#     print("Lengths:", y.batch_sizes)  # Lengths of sequences in the batch
#     break

### model

In [21]:
class GenerativeLSTM(nn.Module):
    def __init__(self, input_dim=20, hidden_dim=128, num_layers=1, dropout=0.3, output_dim=20):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, dropout=dropout if num_layers > 1 else 0)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, packed_input):
        packed_output, _ = self.lstm(packed_input)
        dropped = self.dropout(packed_output.data)
        logits = self.fc(dropped)
        return logits  # shape: [total_timesteps, 20]



### test

In [6]:
model = GenerativeLSTM()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

# Train and eval functions
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for packed_input, packed_target, _ in dataloader:
        inputs = packed_input.to(device)
        targets = torch.argmax(packed_target.data, dim=1).to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for packed_input, packed_target, _ in dataloader:
            inputs = packed_input.to(device)
            targets = torch.argmax(packed_target.data, dim=1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss = evaluate(model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

# Final test evaluation
test_loss = evaluate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}")


Epoch 1/10 - Train Loss: 2.9062 - Val Loss: 2.8592
Epoch 2/10 - Train Loss: 2.8528 - Val Loss: 2.8409
Epoch 3/10 - Train Loss: 2.8234 - Val Loss: 2.8101
Epoch 4/10 - Train Loss: 2.8029 - Val Loss: 2.7917
Epoch 5/10 - Train Loss: 2.7897 - Val Loss: 2.7748
Epoch 6/10 - Train Loss: 2.7756 - Val Loss: 2.7624
Epoch 7/10 - Train Loss: 2.7663 - Val Loss: 2.7500
Epoch 8/10 - Train Loss: 2.7544 - Val Loss: 2.7458
Epoch 9/10 - Train Loss: 2.7522 - Val Loss: 2.7432
Epoch 10/10 - Train Loss: 2.7495 - Val Loss: 2.7287
Test Loss: 2.7355


## Model

In [3]:
from torch.nn.utils.rnn import pad_packed_sequence


class GenerativeLSTM(nn.Module):
    def __init__(self, input_dim=20, hidden_dim=128, num_layers=1, dropout=0.3):
        super(GenerativeLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        # Handle packed input
        if isinstance(x, torch.nn.utils.rnn.PackedSequence):
            packed_output, _ = self.lstm(x)
            unpacked_output, _ = pad_packed_sequence(packed_output, batch_first=True)
            return self.fc(unpacked_output)
        else:
            out, _ = self.lstm(x)
            return self.fc(out)



#### General AMP - train

In [23]:
# Re-import necessary packages after reset
import torch
import torch.nn as nn
import torch.optim as optim
import datetime
import optuna
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
from torch.utils.tensorboard import SummaryWriter


# Criterion

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)


def compute_last_token_loss(output, target_seq, criterion):
    """
    Computes cross-entropy loss on the last time step of each sequence.
    
    Args:
        output: Tensor of shape [B, L, vocab_size]
        target_seq: Tensor of shape [B, L] containing target class indices
    
    Returns:
        loss: Scalar loss computed only on the last token of each sequence
    """
    # Get last time step for each sequence
    last_token_logits = output[:, -1, :]        # [B, vocab_size]
    last_token_targets = target_seq[:, -1, :]      # [B]
    last_token_targets = torch.argmax(last_token_targets, dim=-1)  #  now shape is [batch_size, seq_len]

    # print('last_token_logits',last_token_logits.shape)
    # print('last_token_targets',last_token_targets.shape)

    return criterion(last_token_logits, last_token_targets)

# Training function
def train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=1e-3, weight_decay=1e-4,
                           device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False, train=True):
    model.to(device)
    if not train:
        model.eval()

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    best_val_loss = float('inf')
    log_dir = f"runs-lstm-gen/AMP_LSTM_GEN_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    writer = SummaryWriter(log_dir=log_dir)

    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_loss = 0.0
        for input_seq, target_seq, _ in train_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            optimizer.zero_grad()
            output = model(input_seq)
            # output = output.view(-1, output.shape[-1])
            # output = output.reshape(-1, output.shape[-1])      # [B*L, vocab]
            # output = torch.argmax(output, dim=-1)  # now shape is [batch_size, seq_len]

            
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)
            # # target_seq = target_seq.view(-1)
            # target_seq = torch.argmax(target_seq, dim=-1)  #  now shape is [batch_size, seq_len]

            
            # print('target_shape before reshape',target_seq.shape)
            # target_seq = target_seq.reshape(-1)
            # print(f"Output shape: {output.shape}, Target shape: {target_seq.shape}")

            # loss = criterion(output, target_seq)
            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_train_loss = epoch_loss / len(train_loader)
        val_loss, acc, auc = evaluate_model_generation(model, val_loader, criterion, device, verbose)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        if verbose:
            print(f"Epoch [{epoch}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Acc: {acc:.4f}, AUC: {auc}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if train:
                torch.save(model.state_dict(), 'best_model_lstm_generator.pt')

    writer.close()
    return best_val_loss

# Evaluation function
def evaluate_model_generation(model, data_loader, criterion, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False):
    model.eval()
    all_labels = []
    all_preds = []
    total_loss = 0.0
    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            output = model(input_seq)
            # output = output.view(-1, output.shape[-1])
            # output = output.reshape(-1, output.shape[-1])  # [B*L, vocab]
            # output = torch.argmax(output, dim=-1)  # now shape is [batch_size, seq_len]

            # # target_seq = target_seq.view(-1)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)
            # # target_seq = target_seq.view(-1)
            # # target_seq = target_seq.reshape(-1)
            # # target_seq = target_seq.reshape(-1, target_seq.shape[-1])
            # target_seq = torch.argmax(target_seq, dim=-1)  #  now shape is [batch_size, seq_len]

            # assert output.size(0) == target_seq.size(0), f"Mismatch: {output.size(0)} vs {target_seq.size(0)}"

            # loss = criterion(output, target_seq)
            loss = compute_last_token_loss(output, target_seq, criterion)
            # print('loss done')
            total_loss += loss.item()
            
            
            preds = output[:, -1, :]        # shape: [B, vocab_size]
            preds = torch.argmax(preds, dim=1)  # shape: [B]

            targets = target_seq[:, -1, :]      # shape: [B, vocab_size]
            targets = torch.argmax(targets, dim=-1)  # shape: [B]
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except:
    auc = "undefined"

    return avg_loss, acc, auc

# Objective for Optuna tuning
def objective_generation(trial):
    hidden_dim = trial.suggest_int("hidden_dim", 64, 256)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3)

    model = GenerativeLSTM(hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout)
    val_loss = train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=lr, weight_decay=weight_decay, verbose=False)
    return val_loss

study = optuna.create_study(direction="minimize")
study.optimize(objective_generation, n_trials=20)

lstm_gen_best_params = study.best_trial.params
print(lstm_gen_best_params)

[I 2025-04-23 14:01:33,836] A new study created in memory with name: no-name-ac1e7109-a285-42bc-85d1-80d0c8f1751c
[I 2025-04-23 14:02:14,459] Trial 0 finished with value: 2.352133423089981 and parameters: {'hidden_dim': 193, 'num_layers': 2, 'dropout': 0.16639264160797632, 'lr': 0.0009584674035512115, 'weight_decay': 0.0006112688207040144}. Best is trial 0 with value: 2.352133423089981.
[I 2025-04-23 14:02:53,079] Trial 1 finished with value: 1.9836155623197556 and parameters: {'hidden_dim': 253, 'num_layers': 3, 'dropout': 0.4272420846439058, 'lr': 0.0016812232747644258, 'weight_decay': 0.0004198276532947544}. Best is trial 1 with value: 1.9836155623197556.
[I 2025-04-23 14:03:32,865] Trial 2 finished with value: 0.5449075996875763 and parameters: {'hidden_dim': 117, 'num_layers': 3, 'dropout': 0.3971753738117332, 'lr': 0.005669630280173859, 'weight_decay': 0.0005079920092703994}. Best is trial 2 with value: 0.5449075996875763.
[I 2025-04-23 14:04:06,377] Trial 3 finished with value: 

{'hidden_dim': 101, 'num_layers': 1, 'dropout': 0.11058091088425542, 'lr': 0.009976100969164275, 'weight_decay': 0.00020340520395482203}


In [ ]:
!tensorboard --logdir runs-lstm-gen

#### test

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
import math

# --- Assumes you already have these from your previous steps ---
# lstm_gen_best_params
# train_loader, val_loader, test_loader
# GenerativeLSTM
# compute_last_token_loss

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_final_model(model, train_loader, val_loader, num_epochs=20):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lstm_gen_best_params["lr"], weight_decay=lstm_gen_best_params["weight_decay"])
    writer = SummaryWriter(log_dir=f"runs-lstm-gen/AMPGen_LSTM_final")

    best_val_loss = float('inf')

    for epoch in range(1, num_epochs + 1):
        model.train()
        total_loss = 0.0

        for input_seq, target_seq, _ in train_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            optimizer.zero_grad()

            output = model(input_seq)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        val_loss, acc, auc, perp = evaluate_final_model(model, test_loader)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        print(f"Epoch {epoch}: Train Loss = {avg_train_loss:.4f} | Val Loss = {val_loss:.4f} | Acc = {acc:.4f} | AUC = {auc} | Perplexity = {perp:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # torch.save(model.state_dict(), "best_model_lstm_generator.pt")

    writer.close()
    return model

def evaluate_final_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0.0

    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]  # [B, vocab]
            preds = torch.argmax(preds, dim=1)
            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except Exception:
    auc = "undefined"

    avg_loss = total_loss / len(data_loader)
    perplexity = math.exp(avg_loss)
    return avg_loss, acc, auc, perplexity

# --- Build and train final model using best parameters ---
# lstm_gen_best_params = {'hidden_dim': 133, 'num_layers': 2, 'dropout': 0.10063270147175422, 'lr': 0.003237280156212186, 'weight_decay': 2.2594437829479466e-05}

final_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    dropout=lstm_gen_best_params["dropout"]
)

trained_model = train_final_model(final_model, train_loader, val_loader, num_epochs=20)

# --- Evaluate on test set ---
test_loss, test_acc, test_auc, perp = evaluate_final_model(trained_model, test_loader)
print(f"\n✅ Final Test Metrics:\nLoss = {test_loss:.4f}, Accuracy = {test_acc:.4f}, AUC = {test_auc}, Perplexity = {perp:.4f}")
torch.save(trained_model.state_dict(), 'best_model_lstm_generator_final.pt')


Epoch 1: Train Loss = 2.6271 | Val Loss = 2.2939 | Acc = 0.9877 | AUC = undefined | Perplexity = 9.9133
Epoch 2: Train Loss = 2.0037 | Val Loss = 1.7072 | Acc = 0.9877 | AUC = undefined | Perplexity = 5.5137
Epoch 3: Train Loss = 1.4672 | Val Loss = 1.2243 | Acc = 0.9877 | AUC = undefined | Perplexity = 3.4016
Epoch 4: Train Loss = 1.0436 | Val Loss = 0.8678 | Acc = 0.9857 | AUC = undefined | Perplexity = 2.3817
Epoch 5: Train Loss = 0.7415 | Val Loss = 0.6213 | Acc = 0.9877 | AUC = undefined | Perplexity = 1.8614
Epoch 6: Train Loss = 0.5398 | Val Loss = 0.4616 | Acc = 0.9877 | AUC = undefined | Perplexity = 1.5866
Epoch 7: Train Loss = 0.4091 | Val Loss = 0.3588 | Acc = 0.9877 | AUC = undefined | Perplexity = 1.4316
Epoch 8: Train Loss = 0.3251 | Val Loss = 0.2927 | Acc = 0.9857 | AUC = undefined | Perplexity = 1.3400
Epoch 9: Train Loss = 0.2648 | Val Loss = 0.2521 | Acc = 0.9857 | AUC = undefined | Perplexity = 1.2868
Epoch 10: Train Loss = 0.2229 | Val Loss = 0.2172 | Acc = 0.9857

## tb amp

### data

In [4]:
df = pd.read_csv('../data/all_seq702.csv')
df = df.drop_duplicates(subset='Sequences')

max_length = df['Sequences'].str.len().max()
print(max_length)
# df['Sequences'] = df['Sequences'].apply(lambda x: x.ljust(max_length, 'X'))

unique_letters = set(''.join(df["Sequences"]))
print(unique_letters)
print(len(unique_letters))
amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
non_standard_amino_acids = unique_letters - amino_acids
print(non_standard_amino_acids)
b_count = df["Sequences"].str.count('B').sum()
print(f"Number of 'B' values: {b_count}")
# manually replaced one of the B with D and the other with N

X = df["Sequences"]
y = df["AMP"]

# df_filtered = df[
#     (df['Sequences'].str.len() >= 10) &
#     (df['Sequences'].apply(lambda x: len(set(x)) > 1)) &
#     (~df['Sequences'].str.contains('X'))
# ]
df_filtered = df

def split_sequence(seq, chunk_size=20):
    return [seq[i:i+chunk_size] for i in range(0, len(seq), chunk_size)]

new_rows = []
for _, row in df_filtered.iterrows():
    seq = row['Sequences']
    amp_label = row['AMP']
    if len(seq) > 40:
        for chunk in split_sequence(seq, 20):
            new_rows.append({'Sequences': chunk, 'AMP': amp_label})
    else:
        new_rows.append({'Sequences': seq, 'AMP': amp_label})

df_filtered = pd.DataFrame(new_rows)


df_filtered = df_filtered[
    (df_filtered['Sequences'].str.len() >= 10) &
    (df_filtered['Sequences'].apply(lambda x: len(set(x)) > 1)) &
    (~df_filtered['Sequences'].str.contains('X'))
]
df_filtered = df_filtered[df_filtered['AMP']==1]
df_filtered = df_filtered.drop_duplicates(subset='Sequences')


128
{'L', 'T', 'M', 'E', 'Y', 'R', 'H', 'X', 'W', 'G', 'N', 'K', 'S', 'C', 'P', 'V', 'I', 'Q', 'F', 'D', 'A'}
21
{'X'}
Number of 'B' values: 0


In [5]:
print(min([len(x) for x in df_filtered['Sequences']]))
print(max([len(x) for x in df_filtered['Sequences']]))

10
40


In [6]:
# Re-import libraries after environment reset
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from sklearn.model_selection import train_test_split

# Load data
df = df_filtered

# Clean and inspect
amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
unique_letters = set(''.join(df["Sequences"]))
# non_standard_amino_acids = unique_letters - amino_acids
# df = df[~df["Sequences"].str.contains('|'.join(non_standard_amino_acids))]

# Extract sequences
sequences = df["Sequences"].reset_index(drop=True)

# Define one-hot function
def one_hot_torch(seq: str, dtype=torch.float32):
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    seq_bytes = torch.ByteTensor(list(bytes(seq, "utf-8")))
    aa_bytes = torch.ByteTensor(list(bytes(amino_acids, "utf-8")))
    arr = torch.zeros(len(amino_acids), len(seq_bytes), dtype=dtype)
    for i, aa in enumerate(aa_bytes):
        arr[i, seq_bytes == aa] = 1
    return arr

# Define dataset class
class GenerativeSequenceDataset(Dataset):
    def __init__(self, sequences, one_hot_dtype=torch.float32):
        self.sequences = sequences
        self.one_hot_dtype = one_hot_dtype

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences.iloc[idx]
        input_seq = seq[:-1]
        target_seq = seq[1:]
        length = len(input_seq.replace("X", ""))
        input_one_hot = one_hot_torch(input_seq, dtype=self.one_hot_dtype)
        target_one_hot = one_hot_torch(target_seq, dtype=self.one_hot_dtype)
        return input_one_hot, target_one_hot, length

# Define collate function
def generative_collate_and_pack(batch):
    sequences, targets, lengths = zip(*batch)
    lengths = torch.tensor(lengths)
    sorted_indices = torch.argsort(lengths, descending=True)
    sequences = [sequences[i] for i in sorted_indices]
    targets = [targets[i] for i in sorted_indices]
    lengths = lengths[sorted_indices]
    sequences = [seq.T for seq in sequences]
    targets = [tgt.T for tgt in targets]
    padded_seqs = pad_sequence(sequences, batch_first=False)
    padded_targets = pad_sequence(targets, batch_first=False)
    packed_input = pack_padded_sequence(padded_seqs, lengths.cpu(), batch_first=False)
    packed_target = pack_padded_sequence(padded_targets, lengths.cpu(), batch_first=False)
    return packed_input, packed_target, lengths

# Split and load data
train_seqs, test_seqs = train_test_split(sequences, test_size=0.3, random_state=42)
val_seqs, test_seqs = train_test_split(test_seqs, test_size=0.5, random_state=42)
train_dataset = GenerativeSequenceDataset(train_seqs)
val_dataset = GenerativeSequenceDataset(val_seqs)
test_dataset = GenerativeSequenceDataset(test_seqs)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=generative_collate_and_pack)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=generative_collate_and_pack)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=generative_collate_and_pack)

# Preview batch
batch_sample = next(iter(train_loader))
batch_sample_shapes = {
    "Input shape": batch_sample[0].data.shape,
    "Target shape": batch_sample[1].data.shape,
    "Lengths": batch_sample[0].batch_sizes
}
batch_sample_shapes


{'Input shape': torch.Size([1120, 20]),
 'Target shape': torch.Size([1120, 20]),
 'Lengths': tensor([64, 64, 64, 64, 64, 64, 64, 64, 64, 58, 54, 47, 42, 39, 37, 36, 35, 32,
         30, 16, 15, 14, 14, 11, 10,  8,  8,  8,  7,  4,  4,  4,  4,  3,  1,  1,
          1,  1])}

In [7]:
dataset_sizes = {
    "Train": len(train_dataset),
    "Validation": len(val_dataset),
    "Test": len(test_dataset)
}
print("Dataset sizes:")
for name, size in dataset_sizes.items():
    print(f"{name}: {size}")

Dataset sizes:
Train: 171
Validation: 37
Test: 37


### train for full backprop

In [34]:

# Criterion
PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Transfer Learning Loader
def load_pretrained_weights(model, checkpoint_path):
    pretrained_dict = torch.load(checkpoint_path, map_location='cpu')
    model.load_state_dict(pretrained_dict, strict=False)
    return model

def compute_last_token_loss(output, target_seq, criterion):
    last_token_logits = output[:, -1, :]
    last_token_targets = target_seq[:, -1, :]
    last_token_targets = torch.argmax(last_token_targets, dim=-1)
    return criterion(last_token_logits, last_token_targets)

# Training function
def train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=1e-3, weight_decay=1e-4,
                           device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False, train=True):
    model.to(device)
    if not train:
        model.eval()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    best_val_loss = float('inf')
    log_dir = f"runs-lstm-gen-tb/AMP_LSTM_GEN_TRANSFER_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    writer = SummaryWriter(log_dir=log_dir)

    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_loss = 0.0
        for input_seq, target_seq, _ in train_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            optimizer.zero_grad()
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_train_loss = epoch_loss / len(train_loader)
        val_loss, acc, auc = evaluate_model_generation(model, val_loader, criterion, device, verbose)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        if verbose:
            print(f"Epoch [{epoch}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Acc: {acc:.4f}, AUC: {auc}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # if train:
            #     torch.save(model.state_dict(), 'best_model_lstm_transfer.pt')

    writer.close()
    return best_val_loss

# Evaluation function
def evaluate_model_generation(model, data_loader, criterion, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False):
    model.eval()
    all_labels = []
    all_preds = []
    total_loss = 0.0
    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]
            preds = torch.argmax(preds, dim=1)

            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except:
    auc = "undefined"
    
    return avg_loss, acc, auc

# Optuna objective for fine-tuning
def objective_generation(trial):
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3)

    model = GenerativeLSTM(    
                hidden_dim=lstm_gen_best_params["hidden_dim"],
                num_layers=lstm_gen_best_params["num_layers"],
                dropout=dropout
                # dropout=lstm_gen_best_params["dropout"]
                )
    model = load_pretrained_weights(model, 'best_model_lstm_generator_final.pt')  # path to the general AMP model
    val_loss = train_model_generation(model, train_loader, val_loader, num_epochs=20, lr=lr, weight_decay=weight_decay, verbose=False)
    return val_loss

# Run Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective_generation, n_trials=20)
lstm_gen_best_params_tb = study.best_trial.params
print("Best transfer learning hyperparameters:", lstm_gen_best_params_tb)

[I 2025-04-23 14:30:17,699] A new study created in memory with name: no-name-73cda9ff-9f49-4694-ae74-377f6103161d


[I 2025-04-23 14:30:22,613] Trial 0 finished with value: 0.17065972089767456 and parameters: {'dropout': 0.3980787728630053, 'lr': 0.003262618150370296, 'weight_decay': 0.0007610779642297202}. Best is trial 0 with value: 0.17065972089767456.
[I 2025-04-23 14:30:28,779] Trial 1 finished with value: 0.16326631605625153 and parameters: {'dropout': 0.17500559295476195, 'lr': 0.008779716388199644, 'weight_decay': 0.0006768118069278002}. Best is trial 1 with value: 0.16326631605625153.
[I 2025-04-23 14:30:34,271] Trial 2 finished with value: 0.1712217628955841 and parameters: {'dropout': 0.28931630707511136, 'lr': 0.0056108644742778055, 'weight_decay': 0.00041883086845768166}. Best is trial 1 with value: 0.16326631605625153.
[I 2025-04-23 14:30:38,407] Trial 3 finished with value: 0.1696176379919052 and parameters: {'dropout': 0.3656290382610786, 'lr': 0.0025633203154633794, 'weight_decay': 0.00033162615249359686}. Best is trial 1 with value: 0.16326631605625153.
[I 2025-04-23 14:30:42,514] 

Best transfer learning hyperparameters: {'dropout': 0.17500559295476195, 'lr': 0.008779716388199644, 'weight_decay': 0.0006768118069278002}


#### test

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
import math

# --- Assumes you already have these from your previous steps ---
# lstm_gen_best_params
# train_loader, val_loader, test_loader
# GenerativeLSTM
# compute_last_token_loss

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_final_model(model, train_loader, val_loader, num_epochs=20):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lstm_gen_best_params_tb["lr"], weight_decay=lstm_gen_best_params_tb["weight_decay"])
    writer = SummaryWriter(log_dir=f"runs-lstm-gen-tb/AMPGen_LSTM_final")

    best_val_loss = float('inf')

    for epoch in range(1, num_epochs + 1):
        model.train()
        total_loss = 0.0

        for input_seq, target_seq, _ in train_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            optimizer.zero_grad()

            output = model(input_seq)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        val_loss, acc, auc, perp = evaluate_final_model(model, test_loader)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        print(f"Epoch {epoch}: Train Loss = {avg_train_loss:.4f} | Val Loss = {val_loss:.4f} | Acc = {acc:.4f} | AUC = {auc} | Perplexity = {perp:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # torch.save(model.state_dict(), "best_model_lstm_generator.pt")

    writer.close()
    return model

def evaluate_final_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0.0

    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]  # [B, vocab]
            preds = torch.argmax(preds, dim=1)
            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except Exception:
    auc = "undefined"

    avg_loss = total_loss / len(data_loader)
    perplexity = math.exp(avg_loss)
    return avg_loss, acc, auc, perplexity

# --- Build and train final model using best parameters ---

# lstm_gen_best_params = {'hidden_dim': 133, 'num_layers': 2}

final_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    dropout=lstm_gen_best_params_tb["dropout"],
    # weights_decay=lstm_gen_best_params_tb["weight_decay"],
    # lr=lstm_gen_best_params_tb["lr"]
)

trained_model = train_final_model(final_model, train_loader, val_loader, num_epochs=200)

# --- Evaluate on test set ---
test_loss, test_acc, test_auc, perp = evaluate_final_model(trained_model, test_loader)
print(f" Final Test Metrics:\nLoss = {test_loss:.4f}, Accuracy = {test_acc:.4f}, AUC = {test_auc}, Perplexity = {perp:.4f}")
# Save model weights
torch.save(trained_model.state_dict(), "final_amp_generator_lstm.pt")
print("Model saved to final_amp_generator_lstm.pt")


Epoch 1: Train Loss = 2.9377 | Val Loss = 2.9043 | Acc = 0.0000 | AUC = undefined | Perplexity = 18.2530
Epoch 2: Train Loss = 2.8846 | Val Loss = 2.8489 | Acc = 0.9730 | AUC = undefined | Perplexity = 17.2682
Epoch 3: Train Loss = 2.8364 | Val Loss = 2.7850 | Acc = 0.9730 | AUC = undefined | Perplexity = 16.1997
Epoch 4: Train Loss = 2.7719 | Val Loss = 2.7146 | Acc = 1.0000 | AUC = undefined | Perplexity = 15.0982
Epoch 5: Train Loss = 2.7198 | Val Loss = 2.6705 | Acc = 0.9730 | AUC = undefined | Perplexity = 14.4476
Epoch 6: Train Loss = 2.6647 | Val Loss = 2.6288 | Acc = 0.9730 | AUC = undefined | Perplexity = 13.8569
Epoch 7: Train Loss = 2.6211 | Val Loss = 2.5836 | Acc = 0.9730 | AUC = undefined | Perplexity = 13.2453
Epoch 8: Train Loss = 2.5820 | Val Loss = 2.5400 | Acc = 0.9730 | AUC = undefined | Perplexity = 12.6799
Epoch 9: Train Loss = 2.5216 | Val Loss = 2.4870 | Acc = 0.9730 | AUC = undefined | Perplexity = 12.0257
Epoch 10: Train Loss = 2.4830 | Val Loss = 2.4301 | Acc

#### Generation

In [36]:
import torch
import torch.nn.functional as F
import numpy as np

# Recreate your amino acid vocab
aa_vocab = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_idx = {aa: i for i, aa in enumerate(aa_vocab)}
idx_to_aa = {i: aa for aa, i in aa_to_idx.items()}

def one_hot_encode_amino_acid(aa, vocab=aa_vocab):
    vec = torch.zeros(len(vocab))
    vec[aa_to_idx[aa]] = 1.0
    return vec

def generate_sequence_from_seed(model, seed, max_length=30, temperature=1.0, device='cpu'):
    model.eval()
    input_seq = [one_hot_encode_amino_acid(aa).to(device) for aa in seed]
    input_tensor = torch.stack(input_seq).unsqueeze(0)  # [1, L, 20]

    generated = seed.copy()

    with torch.no_grad():
        for _ in range(max_length - len(seed)):
            output = model(input_tensor)  # [1, L, vocab]
            logits = output[0, -1, :]  # Last time step → [vocab]

            # Apply temperature and sample
            probs = F.softmax(logits / temperature, dim=-1).cpu().numpy()
            next_idx = np.random.choice(len(aa_vocab), p=probs)
            next_aa = idx_to_aa[next_idx]

            # Update sequence
            next_aa_vec = one_hot_encode_amino_acid(next_aa).to(device).unsqueeze(0).unsqueeze(0)  # [1, 1, 20]
            input_tensor = torch.cat([input_tensor, next_aa_vec], dim=1)
            generated.append(next_aa)

    return ''.join(generated)

import numpy as np
from collections import Counter

class LengthSampler:
    def __init__(self, sequence_lengths):
        """
        Initialize sampler from observed sequence lengths.
        
        Args:
            sequence_lengths (list[int]): List of sequence lengths (e.g., [20, 21, 20, 23, ...])
        """
        self.length_counts = Counter(sequence_lengths)
        self.lengths = np.array(sorted(self.length_counts.keys()))
        counts = np.array([self.length_counts[l] for l in self.lengths])
        self.probs = counts / counts.sum()  # Empirical probabilities

    def sample(self, n=1):
        """
        Sample one or more lengths based on the learned distribution.

        Returns:
            np.ndarray of sampled lengths
        """
        return np.random.choice(self.lengths, size=n, p=self.probs)
length_sampler = LengthSampler([len(seq) for seq in df.loc[df['AMP'] == 1, :]['Sequences']])

In [37]:
import random
import numpy as np

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Define the amino acid vocabulary
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def sample_start_amino_acid():
    return random.choice(amino_acids)

# Reload the trained model
gen_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    # dropout=lstm_gen_best_params["dropout"]
)
gen_model.load_state_dict(torch.load("final_amp_generator_lstm.pt"))
gen_model.to(device)
gen_model.eval()
# Define a seed and generate a sequence
sampled_length = length_sampler.sample()[0]
start_aa = sample_start_amino_acid()
seed_sequence = list(start_aa)  # start with valine
generated_peptide = generate_sequence_from_seed(gen_model, seed_sequence, max_length=sampled_length, temperature=1.0, device=device)

print("Generated AMP sequence:", generated_peptide)

Generated AMP sequence: EPAAAAAYPRAYYGGG


#### batch gen

In [ ]:
import random
import numpy as np

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Define the amino acid vocabulary
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def sample_start_amino_acid():
    return random.choice(amino_acids)

# Reload the trained model
gen_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    # dropout=lstm_gen_best_params["dropout"]
)
gen_model.load_state_dict(torch.load("final_amp_generator_lstm.pt"))
gen_model.to(device)
gen_model.eval()
generated_peptides = []
# (Re-run the generation loop to collect sequences)
for x in range(100):
    sampled_length = length_sampler.sample()[0]
    # sampled_length = 20
    start_aa = sample_start_amino_acid()
    seed_sequence = list(start_aa)
    generated_peptide = generate_sequence_from_seed(gen_model, seed_sequence, max_length=sampled_length, temperature=1.2, device=device)
    generated_peptides.append(generated_peptide)
    print("Generated AMP sequence:", generated_peptide)

# Save all generated sequences into a text file
with open("generated_peptides.fasta", "w") as f:
    for i, peptide in enumerate(generated_peptides):
        f.write(f">peptide{i}\n")
        f.write(peptide + "\n")

### frozen

In [236]:

# Criterion
PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Transfer Learning Loader
def load_pretrained_weights(model, checkpoint_path):
    pretrained_dict = torch.load(checkpoint_path, map_location='cpu')
    model.load_state_dict(pretrained_dict, strict=False)
    return model

def compute_last_token_loss(output, target_seq, criterion):
    last_token_logits = output[:, -1, :]
    last_token_targets = target_seq[:, -1, :]
    last_token_targets = torch.argmax(last_token_targets, dim=-1)
    return criterion(last_token_logits, last_token_targets)

def freeze_encoder(model):
    for name, param in model.named_parameters():
        if 'lstm' in name:
            param.requires_grad = False

# Training function
def train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=1e-3, weight_decay=1e-4,
                           device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False, train=True):
    model.to(device)
    if not train:
        model.eval()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    best_val_loss = float('inf')
    log_dir = f"runs-lstm-frozen-gen-tb/AMP_LSTM_GEN_TRANSFER_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    writer = SummaryWriter(log_dir=log_dir)

    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_loss = 0.0
        for input_seq, target_seq, _ in train_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            optimizer.zero_grad()
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_train_loss = epoch_loss / len(train_loader)
        val_loss, acc, auc = evaluate_model_generation(model, val_loader, criterion, device, verbose)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        if verbose:
            print(f"Epoch [{epoch}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Acc: {acc:.4f}, AUC: {auc}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # if train:
            #     torch.save(model.state_dict(), 'best_model_lstm_transfer.pt')

    writer.close()
    return best_val_loss

# Evaluation function
def evaluate_model_generation(model, data_loader, criterion, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False):
    model.eval()
    all_labels = []
    all_preds = []
    total_loss = 0.0
    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]
            preds = torch.argmax(preds, dim=1)

            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except:
    auc = "undefined"
    
    return avg_loss, acc, auc

# Optuna objective for fine-tuning
def objective_generation(trial):
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3)

    model = GenerativeLSTM(    
                hidden_dim=156,
                num_layers=lstm_gen_best_params["num_layers"],
                dropout=lstm_gen_best_params["dropout"])
    model = load_pretrained_weights(model, 'best_model_lstm_generator.pt')  # path to the general AMP model
    freeze_encoder(model)

    val_loss = train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=lr, weight_decay=weight_decay, verbose=False)
    return val_loss

# Run Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective_generation, n_trials=10)
lstm_gen_frozen_best_params_tb = study.best_trial.params
print("Best transfer learning hyperparameters:", lstm_gen_frozen_best_params_tb)

[I 2025-04-22 15:11:24,898] A new study created in memory with name: no-name-cf68f5aa-42bb-4b5b-8b48-a267cfc12f6f
[I 2025-04-22 15:11:31,811] Trial 0 finished with value: 2.1874172687530518 and parameters: {'dropout': 0.26927805101609836, 'lr': 0.006607107825528057, 'weight_decay': 0.0007828517656279971}. Best is trial 0 with value: 2.1874172687530518.
[I 2025-04-22 15:11:37,863] Trial 1 finished with value: 2.287254810333252 and parameters: {'dropout': 0.44973959975333355, 'lr': 0.005780681311997045, 'weight_decay': 0.0007892633251980037}. Best is trial 0 with value: 2.1874172687530518.
[I 2025-04-22 15:11:43,597] Trial 2 finished with value: 2.50271737575531 and parameters: {'dropout': 0.4481068863127341, 'lr': 0.0038894908204979723, 'weight_decay': 0.0007341287032190064}. Best is trial 0 with value: 2.1874172687530518.
[I 2025-04-22 15:11:49,360] Trial 3 finished with value: 2.6780993938446045 and parameters: {'dropout': 0.22541801700207792, 'lr': 0.002533643023326825, 'weight_decay

Best transfer learning hyperparameters: {'dropout': 0.3402136218943675, 'lr': 0.007180611606871151, 'weight_decay': 0.0008128745410642789}


In [237]:
lstm_gen_frozen_best_params_tb

{'dropout': 0.3402136218943675,
 'lr': 0.007180611606871151,
 'weight_decay': 0.0008128745410642789}

In [239]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
import math

# --- Assumes you already have these from your previous steps ---
# lstm_gen_best_params
# train_loader, val_loader, test_loader
# GenerativeLSTM
# compute_last_token_loss

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_final_model(model, train_loader, val_loader, num_epochs=20):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lstm_gen_frozen_best_params_tb["lr"], weight_decay=lstm_gen_frozen_best_params_tb["weight_decay"])
    writer = SummaryWriter(log_dir=f"runs-lstm-frozen-gen-tb/AMPGen_LSTM_final")

    best_val_loss = float('inf')

    for epoch in range(1, num_epochs + 1):
        model.train()
        total_loss = 0.0

        for input_seq, target_seq, _ in train_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            optimizer.zero_grad()

            output = model(input_seq)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        val_loss, acc, auc, perp = evaluate_final_model(model, test_loader)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        print(f"Epoch {epoch}: Train Loss = {avg_train_loss:.4f} | Val Loss = {val_loss:.4f} | Acc = {acc:.4f} | AUC = {auc} | Perplexity = {perp:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # torch.save(model.state_dict(), "best_model_lstm_generator.pt")

    writer.close()
    return model

def evaluate_final_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0.0

    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]  # [B, vocab]
            preds = torch.argmax(preds, dim=1)
            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except Exception:
    auc = "undefined"

    avg_loss = total_loss / len(data_loader)
    perplexity = math.exp(avg_loss)
    return avg_loss, acc, auc, perplexity

# --- Build and train final model using best parameters ---

# lstm_gen_best_params = {'hidden_dim': 133, 'num_layers': 2, 'dropout': 0.10063270147175422, 'lr': 0.003237280156212186, 'weight_decay': 2.2594437829479466e-05}

final_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    dropout=lstm_gen_frozen_best_params_tb["dropout"],
    # weights_decay=lstm_gen_frozen_best_params_tb["weight_decay"],
    # lr=lstm_gen_frozen_best_params_tb["lr"]
)
freeze_encoder(final_model)

trained_model = train_final_model(final_model, train_loader, val_loader, num_epochs=60)

# --- Evaluate on test set ---
test_loss, test_acc, test_auc, perp = evaluate_final_model(trained_model, test_loader)
print(f" Final Test Metrics:\nLoss = {test_loss:.4f}, Accuracy = {test_acc:.4f}, AUC = {test_auc}, Perplexity = {perp:.4f}")
# Save model weights
torch.save(trained_model.state_dict(), "final_amp_frozen_generator_lstm.pt")
print("Model saved to final_amp_generator_lstm.pt")


Epoch 1: Train Loss = 2.9842 | Val Loss = 2.9296 | Acc = 0.9688 | AUC = undefined | Perplexity = 18.7204
Epoch 2: Train Loss = 2.8892 | Val Loss = 2.8361 | Acc = 0.9688 | AUC = undefined | Perplexity = 17.0489
Epoch 3: Train Loss = 2.7940 | Val Loss = 2.7438 | Acc = 0.9688 | AUC = undefined | Perplexity = 15.5452
Epoch 4: Train Loss = 2.7025 | Val Loss = 2.6530 | Acc = 0.9688 | AUC = undefined | Perplexity = 14.1966
Epoch 5: Train Loss = 2.6142 | Val Loss = 2.5641 | Acc = 0.9688 | AUC = undefined | Perplexity = 12.9887
Epoch 6: Train Loss = 2.5228 | Val Loss = 2.4753 | Acc = 0.9688 | AUC = undefined | Perplexity = 11.8852
Epoch 7: Train Loss = 2.4312 | Val Loss = 2.3884 | Acc = 0.9688 | AUC = undefined | Perplexity = 10.8957
Epoch 8: Train Loss = 2.3420 | Val Loss = 2.3028 | Acc = 0.9688 | AUC = undefined | Perplexity = 10.0026
Epoch 9: Train Loss = 2.2545 | Val Loss = 2.2179 | Acc = 0.9688 | AUC = undefined | Perplexity = 9.1885
Epoch 10: Train Loss = 2.1691 | Val Loss = 2.1347 | Acc 

In [240]:
import random
import numpy as np

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Define the amino acid vocabulary
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def sample_start_amino_acid():
    return random.choice(amino_acids)

# Reload the trained model
gen_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    dropout=lstm_gen_best_params["dropout"]
)
gen_model.load_state_dict(torch.load("final_amp_generator_lstm.pt"))
gen_model.to(device)

generated_peptides = []
# (Re-run the generation loop to collect sequences)
for x in range(100):
    sampled_length = length_sampler.sample()[0]
    # sampled_length = 20
    start_aa = sample_start_amino_acid()
    seed_sequence = list(start_aa)
    generated_peptide = generate_sequence_from_seed(gen_model, seed_sequence, max_length=sampled_length, temperature=0.7, device=device)
    generated_peptides.append(generated_peptide)
    print("Generated AMP sequence:", generated_peptide)

# Save all generated sequences into a text file
with open("generated_peptides.fasta", "w") as f:
    for i, peptide in enumerate(generated_peptides):
        f.write(f">peptide{i}\n")
        f.write(peptide + "\n")

Generated AMP sequence: EAAAAAAYRRDYYKGG
Generated AMP sequence: AAAANAKNNYINRG
Generated AMP sequence: KAAWYSKGRNGNGYLRNNPG
Generated AMP sequence: IAPRNYDGFNNNYNNPGYGYYKDYRRYGNGYRNGNNRRY
Generated AMP sequence: IAAKKRNNNDGFRNNYKNSK
Generated AMP sequence: FAASRPYYGY
Generated AMP sequence: EAHAAGNYYANNKGNYNNRN
Generated AMP sequence: VGAAAGARNGNYKGNYKRYKRNRRPGYNGGRRDNKRGRN
Generated AMP sequence: DAAAYYHRYPNKGYYRNNRYYYRGGYRDGRDGPRRK
Generated AMP sequence: WAAINYRPGNKKYNYRYNPN
Generated AMP sequence: QAAANDYYYNDY
Generated AMP sequence: CIRPAKYNGPYRPGRYGN
Generated AMP sequence: AAAGAGYYYYNNYRRYYNNGPGNPNRGGYNG
Generated AMP sequence: DAAAAANNRRYNNYNNGFYY
Generated AMP sequence: HAAAANRRKYRPRNKNVDGG
Generated AMP sequence: IAAAANNGNN
Generated AMP sequence: TAAANNYRGGRFRYPNRNPY
Generated AMP sequence: YGKAAAAGRGNYFYNGR
Generated AMP sequence: AAANADRYYNNYNYYNSSGY
Generated AMP sequence: VAARDAYGNYYPRNNNRSYY
Generated AMP sequence: HAAAGYMGGSR
Generated AMP sequence: VAAARRDGYNN
Genera

In [242]:
torch.cuda.is_available() 

False

# TB no transfer

In [19]:
# Re-import necessary packages after reset
import torch
import torch.nn as nn
import torch.optim as optim
import datetime
import optuna
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
from torch.utils.tensorboard import SummaryWriter


# Criterion

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)


def compute_last_token_loss(output, target_seq, criterion):
    """
    Computes cross-entropy loss on the last time step of each sequence.
    
    Args:
        output: Tensor of shape [B, L, vocab_size]
        target_seq: Tensor of shape [B, L] containing target class indices
    
    Returns:
        loss: Scalar loss computed only on the last token of each sequence
    """
    # Get last time step for each sequence
    last_token_logits = output[:, -1, :]        # [B, vocab_size]
    last_token_targets = target_seq[:, -1, :]      # [B]
    last_token_targets = torch.argmax(last_token_targets, dim=-1)  #  now shape is [batch_size, seq_len]

    # print('last_token_logits',last_token_logits.shape)
    # print('last_token_targets',last_token_targets.shape)

    return criterion(last_token_logits, last_token_targets)

# Training function
def train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=1e-3, weight_decay=1e-4,
                           device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False, train=True):
    model.to(device)
    if not train:
        model.eval()

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    best_val_loss = float('inf')
    log_dir = f"runs-lstm-gen-notrans-tb/AMP_LSTM_GEN_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    writer = SummaryWriter(log_dir=log_dir)

    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_loss = 0.0
        for input_seq, target_seq, _ in train_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            optimizer.zero_grad()
            output = model(input_seq)
            # output = output.view(-1, output.shape[-1])
            # output = output.reshape(-1, output.shape[-1])      # [B*L, vocab]
            # output = torch.argmax(output, dim=-1)  # now shape is [batch_size, seq_len]

            
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)
            # # target_seq = target_seq.view(-1)
            # target_seq = torch.argmax(target_seq, dim=-1)  #  now shape is [batch_size, seq_len]

            
            # print('target_shape before reshape',target_seq.shape)
            # target_seq = target_seq.reshape(-1)
            # print(f"Output shape: {output.shape}, Target shape: {target_seq.shape}")

            # loss = criterion(output, target_seq)
            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_train_loss = epoch_loss / len(train_loader)
        val_loss, acc, auc = evaluate_model_generation(model, val_loader, criterion, device, verbose)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        if verbose:
            print(f"Epoch [{epoch}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Acc: {acc:.4f}, AUC: {auc}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if train:
                torch.save(model.state_dict(), 'best_model_lstm_generator-notrans-tb.pt')

    writer.close()
    return best_val_loss

# Evaluation function
def evaluate_model_generation(model, data_loader, criterion, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False):
    model.eval()
    all_labels = []
    all_preds = []
    total_loss = 0.0
    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            output = model(input_seq)
            # output = output.view(-1, output.shape[-1])
            # output = output.reshape(-1, output.shape[-1])  # [B*L, vocab]
            # output = torch.argmax(output, dim=-1)  # now shape is [batch_size, seq_len]

            # # target_seq = target_seq.view(-1)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)
            # # target_seq = target_seq.view(-1)
            # # target_seq = target_seq.reshape(-1)
            # # target_seq = target_seq.reshape(-1, target_seq.shape[-1])
            # target_seq = torch.argmax(target_seq, dim=-1)  #  now shape is [batch_size, seq_len]

            # assert output.size(0) == target_seq.size(0), f"Mismatch: {output.size(0)} vs {target_seq.size(0)}"

            # loss = criterion(output, target_seq)
            loss = compute_last_token_loss(output, target_seq, criterion)
            # print('loss done')
            total_loss += loss.item()
            
            
            preds = output[:, -1, :]        # shape: [B, vocab_size]
            preds = torch.argmax(preds, dim=1)  # shape: [B]

            targets = target_seq[:, -1, :]      # shape: [B, vocab_size]
            targets = torch.argmax(targets, dim=-1)  # shape: [B]
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except:
    auc = "undefined"

    return avg_loss, acc, auc

# Objective for Optuna tuning
def objective_generation(trial):
    hidden_dim = trial.suggest_int("hidden_dim", 64, 256)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3)

    model = GenerativeLSTM(hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout)
    val_loss = train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=lr, weight_decay=weight_decay, verbose=False)
    return val_loss

study = optuna.create_study(direction="minimize")
study.optimize(objective_generation, n_trials=20)

lstm_gen_notrans_tb_best_params = study.best_trial.params
print(lstm_gen_notrans_tb_best_params)

[I 2025-04-25 12:35:22,470] A new study created in memory with name: no-name-6f159489-f081-4a98-a9ee-c88b00a56ada


[I 2025-04-25 12:35:25,425] Trial 0 finished with value: 2.824002265930176 and parameters: {'hidden_dim': 206, 'num_layers': 2, 'dropout': 0.4839790848775959, 'lr': 0.003644623413660024, 'weight_decay': 0.0008083199139350111}. Best is trial 0 with value: 2.824002265930176.
[I 2025-04-25 12:35:28,158] Trial 1 finished with value: 2.824535846710205 and parameters: {'hidden_dim': 139, 'num_layers': 2, 'dropout': 0.4424040668201631, 'lr': 0.003142478903807537, 'weight_decay': 3.501252661638913e-05}. Best is trial 0 with value: 2.824002265930176.
[I 2025-04-25 12:35:31,051] Trial 2 finished with value: 2.6291379928588867 and parameters: {'hidden_dim': 192, 'num_layers': 2, 'dropout': 0.4309368762499072, 'lr': 0.008735760571752748, 'weight_decay': 0.0005650464565859144}. Best is trial 2 with value: 2.6291379928588867.
[I 2025-04-25 12:35:33,743] Trial 3 finished with value: 2.846959352493286 and parameters: {'hidden_dim': 111, 'num_layers': 2, 'dropout': 0.44656884698028143, 'lr': 0.00205705

{'hidden_dim': 254, 'num_layers': 1, 'dropout': 0.10699047044012125, 'lr': 0.009935448382765784, 'weight_decay': 0.0009603867104781691}


### test

In [21]:
lstm_gen_notrans_tb_best_params

{'hidden_dim': 254,
 'num_layers': 1,
 'dropout': 0.10699047044012125,
 'lr': 0.009935448382765784,
 'weight_decay': 0.0009603867104781691}

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
import math

# --- Assumes you already have these from your previous steps ---
# lstm_gen_best_params
# train_loader, val_loader, test_loader
# GenerativeLSTM
# compute_last_token_loss

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_final_model(model, train_loader, val_loader, num_epochs=20):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lstm_gen_notrans_tb_best_params["lr"], weight_decay=lstm_gen_notrans_tb_best_params["weight_decay"])
    writer = SummaryWriter(log_dir=f"runs-lstm-gen-notrans-tb/AMPGen_LSTM_final")

    best_val_loss = float('inf')

    for epoch in range(1, num_epochs + 1):
        model.train()
        total_loss = 0.0

        for input_seq, target_seq, _ in train_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            optimizer.zero_grad()

            output = model(input_seq)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        val_loss, acc, auc, perp = evaluate_final_model(model, test_loader)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        print(f"Epoch {epoch}: Train Loss = {avg_train_loss:.4f} | Val Loss = {val_loss:.4f} | Acc = {acc:.4f} | AUC = {auc} | Perplexity = {perp:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # torch.save(model.state_dict(), "best_model_lstm_generator.pt")

    writer.close()
    return model

def evaluate_final_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0.0

    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]  # [B, vocab]
            preds = torch.argmax(preds, dim=1)
            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except Exception:
    auc = "undefined"

    avg_loss = total_loss / len(data_loader)
    perplexity = math.exp(avg_loss)
    return avg_loss, acc, auc, perplexity

# --- Build and train final model using best parameters ---

final_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_notrans_tb_best_params["hidden_dim"],
    num_layers=lstm_gen_notrans_tb_best_params["num_layers"],
    dropout=lstm_gen_notrans_tb_best_params["dropout"]
)

trained_model = train_final_model(final_model, train_loader, val_loader, num_epochs=60)

# --- Evaluate on test set ---
test_loss, test_acc, test_auc, perp = evaluate_final_model(trained_model, test_loader)
print(f"\n✅ Final Test Metrics:\nLoss = {test_loss:.4f}, Accuracy = {test_acc:.4f}, AUC = {test_auc}, Perplexity = {perp:.4f}")


Epoch 1: Train Loss = 2.9754 | Val Loss = 2.9310 | Acc = 1.0000 | AUC = undefined | Perplexity = 18.7457
Epoch 2: Train Loss = 2.9193 | Val Loss = 2.8671 | Acc = 1.0000 | AUC = undefined | Perplexity = 17.5852
Epoch 3: Train Loss = 2.8558 | Val Loss = 2.7980 | Acc = 1.0000 | AUC = undefined | Perplexity = 16.4116
Epoch 4: Train Loss = 2.7848 | Val Loss = 2.7141 | Acc = 1.0000 | AUC = undefined | Perplexity = 15.0915
Epoch 5: Train Loss = 2.7334 | Val Loss = 2.6830 | Acc = 0.9730 | AUC = undefined | Perplexity = 14.6288
Epoch 6: Train Loss = 2.6741 | Val Loss = 2.6266 | Acc = 1.0000 | AUC = undefined | Perplexity = 13.8272
Epoch 7: Train Loss = 2.6196 | Val Loss = 2.5721 | Acc = 0.9730 | AUC = undefined | Perplexity = 13.0932
Epoch 8: Train Loss = 2.5938 | Val Loss = 2.5299 | Acc = 0.9730 | AUC = undefined | Perplexity = 12.5527
Epoch 9: Train Loss = 2.5239 | Val Loss = 2.4879 | Acc = 0.9730 | AUC = undefined | Perplexity = 12.0358
Epoch 10: Train Loss = 2.4682 | Val Loss = 2.4253 | Acc

### batch gen

In [23]:
import random
import numpy as np
from collections import Counter
import torch
import torch.nn.functional as F
import numpy as np

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# final_model = GenerativeLSTM(
#     input_dim=20,
#     hidden_dim=lstm_gen_notrans_tb_best_params["hidden_dim"],
#     num_layers=lstm_gen_notrans_tb_best_params["num_layers"],
#     dropout=lstm_gen_notrans_tb_best_params["dropout"]
# )
# trained_model = GenerativeLSTM(
#     input_dim=20,
#     hidden_dim=176,
#     num_layers=lstm_gen_notrans_tb_best_params["num_layers"],
#     dropout=lstm_gen_notrans_tb_best_params["dropout"]
# )


# final_model.load_state_dict(torch.load("best_model_lstm_generator-notrans-tb.pt"))

# Define the amino acid vocabulary
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def sample_start_amino_acid():
    return random.choice(amino_acids)

class LengthSampler:
    def __init__(self, sequence_lengths):
        """
        Initialize sampler from observed sequence lengths.
        
        Args:
            sequence_lengths (list[int]): List of sequence lengths (e.g., [20, 21, 20, 23, ...])
        """
        self.length_counts = Counter(sequence_lengths)
        self.lengths = np.array(sorted(self.length_counts.keys()))
        counts = np.array([self.length_counts[l] for l in self.lengths])
        self.probs = counts / counts.sum()  # Empirical probabilities

    def sample(self, n=1):
        """
        Sample one or more lengths based on the learned distribution.

        Returns:
            np.ndarray of sampled lengths
        """
        return np.random.choice(self.lengths, size=n, p=self.probs)
length_sampler = LengthSampler([len(seq) for seq in df.loc[df['AMP'] == 1, :]['Sequences']])


# Recreate your amino acid vocab
aa_vocab = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_idx = {aa: i for i, aa in enumerate(aa_vocab)}
idx_to_aa = {i: aa for aa, i in aa_to_idx.items()}

def one_hot_encode_amino_acid(aa, vocab=aa_vocab):
    vec = torch.zeros(len(vocab))
    vec[aa_to_idx[aa]] = 1.0
    return vec

def generate_sequence_from_seed(model, seed, max_length=30, temperature=1.0, device='cpu'):
    model.eval()
    input_seq = [one_hot_encode_amino_acid(aa).to(device) for aa in seed]
    input_tensor = torch.stack(input_seq).unsqueeze(0)  # [1, L, 20]

    generated = seed.copy()

    with torch.no_grad():
        for _ in range(max_length - len(seed)):
            output = model(input_tensor)  # [1, L, vocab]
            logits = output[0, -1, :]  # Last time step → [vocab]

            # Apply temperature and sample
            probs = F.softmax(logits / temperature, dim=-1).cpu().numpy()
            next_idx = np.random.choice(len(aa_vocab), p=probs)
            next_aa = idx_to_aa[next_idx]

            # Update sequence
            next_aa_vec = one_hot_encode_amino_acid(next_aa).to(device).unsqueeze(0).unsqueeze(0)  # [1, 1, 20]
            input_tensor = torch.cat([input_tensor, next_aa_vec], dim=1)
            generated.append(next_aa)

    return ''.join(generated)

# Reload the trained model
gen_model = trained_model
gen_model.to(device)

generated_peptides = []
# (Re-run the generation loop to collect sequences)
for x in range(100):
    sampled_length = length_sampler.sample()[0]
    # sampled_length = 20
    start_aa = sample_start_amino_acid()
    seed_sequence = list(start_aa)
    generated_peptide = generate_sequence_from_seed(gen_model, seed_sequence, max_length=sampled_length, temperature=0.7, device=device)
    generated_peptides.append(generated_peptide)
    print("Generated AMP sequence:", generated_peptide)

# Save all generated sequences into a text file
with open("generated_peptides-notrans.fasta", "w") as f:
    for i, peptide in enumerate(generated_peptides):
        f.write(f">peptide{i}\n")
        f.write(peptide + "\n")

Generated AMP sequence: EVNKAFAYPRAYWGGG
Generated AMP sequence: AACCLDGKKSGKPA
Generated AMP sequence: KAAYYSHDPKGKAYGPHKKG
Generated AMP sequence: IIWWLYAGAHKHVKHKGSEYSGATRSSEKGYPIDHHSPY
Generated AMP sequence: IALQKSKKKAGAPHKYGKS
Generated AMP sequence: FAAAYSPYSGYK
Generated AMP sequence: ERAAGKSYAKKGGKYHKRKYYGKHHA
Generated AMP sequence: VAAEYGGKYGPSGSKPPKFW
Generated AMP sequence: DAAKPAKGPGQKYG
Generated AMP sequence: WAWTGPSKKGFYYP
Generated AMP sequence: QAMWYSPCGYPAGP
Generated AMP sequence: CAGPPGRGHS
Generated AMP sequence: APKKAHGGYKYPSKNKGSHA
Generated AMP sequence: DAYYYHAYKYYYHKYHGKYQ
Generated AMP sequence: HAHYAKYSQQKHSSYYKKSP
Generated AMP sequence: IKSFHALAKKHPAAVKGKSG
Generated AMP sequence: TAAKKPRYKHSHKEAYYQKG
Generated AMP sequence: YAGPPGYSKPKG
Generated AMP sequence: AHAAAAYQKFKKGKK
Generated AMP sequence: VAAGPKYPGDPAPYNKPKKY
Generated AMP sequence: HVTAAAAEPDHYASHG
Generated AMP sequence: VASPSGGSSYKKSKYYKSSG
Generated AMP sequence: QARGYKAYEHYYMPKHHPSSSFKCK